<a href="https://colab.research.google.com/github/farfaness/Scraping/blob/master/pap_web_scraping_selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Selenium : Mets ton navigateur en mode automatique


![alt text](https://etcoe.in/news-blog/wp-content/uploads/2018/03/selenium.jpg?w=1200)

La doc : https://selenium-python.readthedocs.io/navigating.html<br>
cheatsheet : http://allselenium.info/python-selenium-commands-cheat-sheet-frequently-used/<br>
moins complète mais plus propre : http://akul.me/blog/2016/selenium-cheatsheet/ <br>

## Installation 

Avec Anaconda : ouvrir anaconda prompt et executer : "conda install -c conda-forge selenium" 
Avec juste python, télécharger geckodrive, le mettre dans le path ou au même endroit que votre jupyter notebook
https://github.com/mozilla/geckodriver/releases

In [0]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains


#browser = webdriver.Firefox()
# browser = webdriver.Firefox(executable_path='C:/path/to/geckodriver.exe') au cas où geckdriver.exe n'est pas dans le path ni au même endroit
browser = webdriver.Chrome()
browser.get('https://www.pap.fr/')

In [0]:
browser.maximize_window()

In [0]:
#Ouvre menu défilant (Pour choisir location d'appart)
button_choix = browser.find_element_by_class_name("CaptionCont.SelectBox") 
button_choix.click()

In [0]:
#Clique sur location dans le menu défilant
button_choix_options = browser.find_elements_by_class_name('opt')
button_choix_options[1].click()

In [0]:
# Tape paris dans barre de recherche 
recherche = browser.find_element_by_id('token-input-geo_objets_ids') #barre recherche
recherche.send_keys("paris(75)") #rentre paris(75) dans la barre de recherche

In [0]:
#Clique sur okay (pour valider le paris(75) dans menu défilant de la barre de recherche)
actions = ActionChains(browser)
actions.send_keys(Keys.ENTER).perform() 

In [0]:
#lance la recherche en cliquant sur bouton recherche
button_recherche = browser.find_element_by_class_name("btn.btn-full-width")
button_recherche.click() 

In [0]:
#Prendre appartements en type de bien
button_choix = browser.find_element_by_class_name("CaptionCont.SelectBox") #menu déroullant
button_choix.click()

button_choix_options = browser.find_elements_by_class_name('opt') #choix de appartements
button_choix_options[1].click()

In [0]:
# Actualiser 
button_actualiser = browser.find_element_by_class_name('form-group') #cliquer sur actualiser 
button_actualiser.click()

In [0]:
# choisir appartements entre 13 et 35 m2
surface_min = browser.find_element_by_id("surface_min") # surface min
surface_min.send_keys("13") 

surface_max = browser.find_element_by_id("surface_max") # surface max
surface_max.send_keys("35") 

In [0]:
# choisir appartements entre 500 et 650 euros
prix_min = browser.find_element_by_id("prix_min") # prix min
prix_min.send_keys("500") 

prix_max = browser.find_element_by_id("prix_max") # prix max
prix_max.send_keys("650") 

In [0]:
# Actualiser 
button_actualiser = browser.find_element_by_class_name('form-group') #cliquer sur actualiser 
button_actualiser.click()

In [0]:
# Actualiser (à cause du menu déroullant il faut actualiser 2 fois)
button_actualiser = browser.find_element_by_class_name('form-group') #cliquer sur actualiser 
button_actualiser.click()

In [0]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import pandas as pd
import time

def check_exists_by_class_name(class_name):
    try:
        browser.find_element_by_class_name(class_name)
    except NoSuchElementException:
        return False
    return True

# Dataframe à updater avec dico infos appart
dataframe = pd.DataFrame(columns = ["reference_bien", "description_bien", "prix_bien", "arrondissement_bien", "metros_bien", "description_complete_bien"])

while True:
    tot_appart = len(browser.find_elements_by_class_name('h1'))
    nb_appart = 0

    while nb_appart < tot_appart : # Tant qu'il reste des apparts sur la page

        try :    
            # liste des apparts à  faire
            time.sleep(1)
            liste_appart = browser.find_elements_by_class_name('h1')
            
            #Scroll et Click sur l'appart
            target = liste_appart[nb_appart]
            actions = ActionChains(browser)
            actions.move_to_element(target)
            actions.perform()
            target.click()

            #Récupére les infos sur l'appart dans un dico
            d = {}
            d["reference_bien"] = browser.find_element_by_class_name('item-date').text
            d["description_bien"] = browser.find_element_by_class_name('item-title').text
            d["prix_bien"] = browser.find_element_by_class_name('item-price').text
            d["arrondissement_bien"] = browser.find_element_by_class_name('margin-bottom-8').text
            d["description_complete_bien"] = browser.find_element_by_class_name('margin-bottom-30').text

            metros = browser.find_elements_by_class_name('item-transports')
            d["metros_bien"] = [metros.text for metros in metros]

            #Update du dataframe avec le dico
            dataframe = dataframe.append(d, ignore_index=True)
            nb_appart = nb_appart +1

            #Revient sur la page d'avant
            browser.back() 

        except ElementClickInterceptedException: 
            bouton_close = browser.find_element_by_class_name('ui-button.ui-widget.ui-state-default.ui-corner-all.ui-button-icon-only.ui-dialog-titlebar-close')
            bouton_close.click() #close the pop up

    if not check_exists_by_class_name("next") : 
        break #Arrete lorsqu'il est sur la dernière page, après dernier appartement

    else :
        try : 
            # Si ce n'est pas la dernière page: Scroll to the end of the page and click on next
            target = browser.find_element_by_class_name('cover.cover-bg-9.margin-bottom-30') #elément en fin de page
            actions = ActionChains(browser)
            actions.move_to_element(target)
            actions.perform() #scroll jusqu'à cet élément

            # Click on next bouton
            bouton_next = browser.find_element_by_class_name('next')
            bouton_next.click()
        
        except ElementClickInterceptedException: 
            bouton_close = browser.find_element_by_class_name('ui-button.ui-widget.ui-state-default.ui-corner-all.ui-button-icon-only.ui-dialog-titlebar-close')
            bouton_close.click() #close the pop up
            
             # Si ce n'est pas la dernière page: Scroll to the end of the page and click on next
            target = browser.find_element_by_class_name('cover.cover-bg-9.margin-bottom-30') #elément en fin de page
            actions = ActionChains(browser)
            actions.move_to_element(target)
            actions.perform() #scroll jusqu'à cet élément

            # Click on next bouton
            bouton_next = browser.find_element_by_class_name('next')
            bouton_next.click()
                       

# Utilise reference du bien comme référence et supprime les duplicate (garde première occurence) 
dataframe.set_index('reference_bien', inplace = True)
dataframe_test = dataframe[~dataframe.index.duplicated(keep='first')] #Pas de duplicate pour les index
# methode qui invert les booleans series ou dataframe
# Attention methode qui fonctionne que pour objects du type np.dtype(object) = bool, cad Series ou Dataframe !!!
# Sinon il inversera les bits : en faisant -(n+1)

# possibilité bis : df3.reset_index().drop_duplicates(subset='index', keep='first').set_index('index')
# possibilité ter : df3.groupby(df3.index).first()
# with import timeit %timeit la première est la plus rapide


In [0]:
len(dataframe_test)

178

In [0]:
dataframe_test.head()

,description_bien,prix_bien,arrondissement_bien,metros_bien,description_complete_bien
reference_bien,,,,,
Réf. : C75/0756 / 21 juin 2019,Location appartement 2 pièces 50 m² Paris 15E\...,1.500 €,Paris 15e (75015),"[, , ]",Paris 15e (75015)\n2 pièces 1 chambre 50 m²\nV...
Réf. : C54/1828 / 21 juin 2019,"Location garage, parking Paris 14E\n95 €",95 €,Paris 14e (75014),"[Mouton-Duvernet, Alésia, Pernety]",Paris 14e (75014)\nVoir l'adresse du bien sur ...
Réf. : C74/0790 / 21 juin 2019,Location meublée studio 17 m² Paris 16E\n780 €,780 €,Paris 16e (75016),"[Passy, Champ de Mars - Tour Eiffel, Bir-Hakei...",Paris 16e (75016)\n1 pièce 1 chambre 17 m²\nVo...
Réf. : C66/0467 / 21 juin 2019,Location studio 35 m² Paris 15E\n900 €,900 €,Paris 15e (75015),"[, , ]",Paris 15e (75015)\n1 pièce 35 m²\nVoir la comm...
Réf. : C67/1452 / 21 juin 2019,Location meublée appartement 2 pièces 70 m² Pa...,2.100 €,Paris 11e (75011),"[Voltaire (Léon Blum), Saint-Ambroise, Rue Sai...",Paris 11e (75011)\n2 pièces 1 chambre 70 m²\nV...


In [0]:
#Save the dataframe as csv
dataframe_test.to_csv('pap_scraping_200.csv')

################ INFOS ######################"

**Pour récupérer les informations text d'une balise, ajouté .text à un objet selenim**

In [0]:
numero_s = browser.find_element_by_xpath('//*[@id="bi-bloc-065153164000000000C0001"]/div[2]/header/div[1]/div/h2/a')
numero_s #objet Selenium

In [0]:
numero.text # récupère sa valeur

**Pour récupérer leurs valeurs, deux manières :** <br>
La débutante : 

In [0]:
liste_valeur = []
for numero in numeros:
    liste_valeur.append(numero.text)
liste_valeur

la plus directe : 

In [0]:
liste_valeur = [numero.text for numero in numeros]
liste_valeur